# Setup - Task 1 Colab Enterprise Notebook
* Note that for the pip install, I will use an alternative code segment which worked in mid-Oct 2025 - https://github.com/KaiquanMah/Training/blob/master/Google%20Cloud/Vertex/day4-Leverage%20Firestore%20as%20a%20Vector%20Store/cymbal_ingest_to_vector_database.ipynb
* Original code segment from 2025.10.23 Google Qwiklab challenge lab failed but support was unable to offer help
```
!pip install --quiet --upgrade google-cloud-logging google_cloud_firestore google_cloud_aiplatform langchain langchain-google-vertexai langchain_community langchain_experimental pymupdf
```

In [1]:
!pip install --quiet --upgrade google-cloud-logging \
google_cloud_firestore \
google_cloud_aiplatform \
langchain \
langchain-google-vertexai \
langchain_community \
langchain_experimental \
pymupdf \
requests==2.32.3 \
protobuf==5.29.5


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.8/449.8 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import vertexai
import logging
import google.cloud.logging
from vertexai.language_models import TextEmbeddingModel
from vertexai.generative_models import GenerativeModel

import pickle
from IPython.display import display, Markdown

from langchain_google_vertexai import VertexAIEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_experimental.text_splitter import SemanticChunker

from google.cloud import firestore
from google.cloud.firestore_v1.vector import Vector
from google.cloud.firestore_v1.base_vector_query import DistanceMeasure

In [2]:
PROJECT_ID = "qwiklabs-gcp-01-32c593469378"
LOCATION = "us-central1"
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [3]:
embedding_model = VertexAIEmbeddings(model_name="text-embedding-005")

/usr/local/lib/python3.11/dist-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


# Task 2. Download, process and chunk data semantically

In this section, you will prepare the NYC Food Safety Manual for Retrieval-Augmented Generation (RAG). Clean the PDF content and split it into meaningful chunks based on semantic similarity using sentence embeddings and generate numerical representations (embeddings) for each identified text chunk.



In [4]:
# Download the New York City Department of Health and Mental Hygiene's Food Protection Training Manual. This document will serve as your Retrieval-Augmented Generation source content.

!gcloud storage cp gs://partner-genai-bucket/genai069/nyc_food_safety_manual.pdf .

Copying gs://partner-genai-bucket/genai069/nyc_food_safety_manual.pdf to file://./nyc_food_safety_manual.pdf

Average throughput: 60.6MiB/s


In [6]:
# Use the LangChain class PyMuPDFLoader to load the contents of the PDF to a variable named 'data'.
# https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/pdf/#using-pymupdf

loader = PyMuPDFLoader("nyc_food_safety_manual.pdf")
data = loader.load()


In [8]:
# The following function is provided to do some basic cleaning on artifacts found in this particular document. Create a variable called cleaned_pages that is a list of strings, with each string being a page of content cleaned by this function.
def clean_page(page):
  return page.page_content.replace("-\n","")\
                          .replace("\n"," ")\
                          .replace("\x02","")\
                          .replace("\x03","")\
                          .replace("fo d P R O T E C T I O N  T R A I N I N G  M A N U A L","")\
                          .replace("N E W  Y O R K  C I T Y  D E P A R T M E N T  O F  H E A L T H  &  M E N T A L  H Y G I E N E","")

cleaned_pages = [clean_page(page) for page in data]

Use LangChain's SemanticChunker with the embedding_model you created earlier to split the first five pages of cleaned_pages into text chunks. The SemanticChunker determines when to start a new chunk when it encounters a larger distance between sentence embeddings. Save the strings of page content from the resulting documents into a list of strings called chunked_content. Take a look at a few of the chunks to get familiar with the content.
https://python.langchain.com/v0.2/docs/how_to/semantic-chunker/#create-text-splitter


In [9]:
text_splitter = SemanticChunker(embedding_model)
chunked_content = text_splitter.create_documents(cleaned_pages[0:5])
chunked_content = [chunk.page_content for chunk in chunked_content if chunk.page_content]


In [11]:
display(chunked_content[:5])


['The Health Code These are regulations that were formulated to allow the  Department to effectively protect the health of the population. Among the rules embodied in the Health Code is Article 81 which regulates the operations of food establishments for the purpose of preventing public health hazards. Environmental Health Division  The Division of Environmental Health is the Commission within the Health Department that is concerned with public health and works to eliminate the incidence of injury and illness caused by environmental factors. There are several Offices and Bureaus within this division. One of these is the Bureau of Food Safety and Community Sanitation that has the responsibility for conducting inspections of food service and food processing establishments. These inspections are performed by Public Health Sanitarians. Anti-corruption Warning All Sanitarians have Department of Health and Mental Hygiene badges and identification cards which they must display whenever it is 

Use the embedding_model to generate embeddings of the text chunks, saving them to a list called chunked_embeddings. To do so, pass your list of chunks to the VertexAIEmbeddings class's embed_documents() method.
https://python.langchain.com/v0.2/docs/integrations/text_embedding/google_vertex_ai_palm/


In [12]:
chunked_embeddings = embedding_model.embed_documents(chunked_content)


In [15]:
# 768 dims
len(chunked_embeddings[:1][0])

768

In [13]:
display(chunked_embeddings[:1])


[[-0.04205901920795441,
  -0.018308516591787338,
  -0.01328700315207243,
  -0.05176236853003502,
  0.016785096377134323,
  0.034466180950403214,
  0.013948620297014713,
  0.02817848138511181,
  0.02424997091293335,
  0.009253749623894691,
  -0.05151556432247162,
  -0.0885138213634491,
  -0.00404269527643919,
  -0.10059984028339386,
  0.019663695245981216,
  -0.039846841245889664,
  0.0046494207344949245,
  -0.04914997145533562,
  -0.007427764590829611,
  0.03015778958797455,
  -0.008879957720637321,
  -0.062279995530843735,
  0.0011091545457020402,
  -0.011204574257135391,
  -0.03136909008026123,
  0.010546463541686535,
  -0.003176371566951275,
  -0.0329093374311924,
  -0.028883982449769974,
  -0.024877125397324562,
  0.024422328919172287,
  0.05332880839705467,
  0.0789521336555481,
  -0.03262040391564369,
  0.01957697980105877,
  -0.026079880073666573,
  0.036542996764183044,
  0.007938552647829056,
  0.04648442193865776,
  0.010396458208560944,
  0.03674811124801636,
  0.00399580504

In [16]:
# You should have successfully chunked & embedded a short section of the document. To get the chunks & corresponding embeddings for the full document, run the following code:

!gcloud storage cp gs://partner-genai-bucket/genai069/chunked_content.pkl .
!gcloud storage cp gs://partner-genai-bucket/genai069/chunked_embeddings.pkl .

chunked_content = pickle.load(open("chunked_content.pkl", "rb"))
chunked_embeddings = pickle.load(open("chunked_embeddings.pkl", "rb"))

# Do not delete this logging statement.
client = google.cloud.logging.Client()
client.setup_logging()
log_message = f"chunked contents are: {chunked_content[0][:20]}"
logging.info(log_message)


Copying gs://partner-genai-bucket/genai069/chunked_content.pkl to file://./chunked_content.pkl
Copying gs://partner-genai-bucket/genai069/chunked_embeddings.pkl to file://./chunked_embeddings.pkl

Average throughput: 165.4MiB/s


INFO:root:chunked contents are: The Health Code Thes


# Task 3. Prepare your vector database
In this section, you will set up a Firestore database to store the processed NYC Food Safety Manual chunks and their embeddings for efficient retrieval. You'll then build a search function to find relevant information based on a user query.

(Refer back to https://partner.skills.google/course_templates/1322/labs/550578)


Create a Firestore database with the default name of (default) in Native Mode and leave the other settings to default.




Next, in your Colab Enterprise Notebook populate a db variable with a Firestore Client.

Use a variable called collection to create a reference to a collection named food-safety.

Using a combination of your lists chunked_content and chunked_embeddings, add a document to your collection for each of your chunked documents. Each document can be assigned a random ID, but it should have a field called content to store the chunk text and a field called embedding to store a Firestore Vector() of the associated embedding.


In [18]:
# Initialize Firestore client
db = firestore.Client(project=PROJECT_ID)
collection = db.collection("food-safety")

# Store each embedding and chunk
pairs = zip(chunked_content, chunked_embeddings)

for content, embedding in pairs:
    doc = {
        "chunk": content,
        "embedding": Vector(embedding)
    }
    collection.add(doc)

Complete the function below to receive a query, get its embedding, and compile a context consisting of the text from the 5 documents with the most similar embeddings. This time, use the embed_query() method of the LangChain VertexAIEmbeddings embedding_model to embed the user's query.
https://python.langchain.com/v0.2/docs/integrations/text_embedding/google_vertex_ai_palm/#embed-single-texts
```
def search_vector_database(query: str):

  context = ""

  # 1. Generate the embedding of the query

  # 2. Get the 5 nearest neighbors from your collection.
  # Call the get() method on the result of your call to
  # find_nearest to retrieve document snapshots.

  # 3. Call to_dict() on each snapshot to load its data.
  # Combine the snapshots into a single string named context


  return context
```

In [19]:
# embed a user query, retrieve the most relevant chunks and generate an answer
def search_vector_database(query: str):
    """
    Performs a vector search in Firestore to find the top_k most relevant chunks.
    """
    # 1. Generate the embedding for the query.
    emb = embedding_model.embed_query(query)

    # 2. Get the 5 nearest neighbors from your collection.
    # Call the get() method on the result of your call to
    # find_nearest to retrieve document snapshots.

    # hardcode k=5 for 5 nearest neighbours
    k = 5
    neighbors = collection.find_nearest(
        vector_field="embedding",
        query_vector=Vector(emb),
        distance_measure=DistanceMeasure.COSINE,
        limit=k
    ).get()

    # 3. Extract the 'chunk' field from each document.
    # Call to_dict() on each snapshot to load its data.
    # Combine the snapshots into a single string named context
    context = "\n\n".join([neighbor.to_dict()["chunk"] for neighbor in neighbors])

    return context



When you first run this function using the code below, it will fail, but it will provide you a gcloud command to generate a Firestore index. Run that gcloud command to create a vector index for your collection using your embedding field.


In [20]:
# retrieve the most relevant content chunks from Firestore, use those chunks as context to generate an answer using the Gemini model

%%capture --no-stdout

query = "What should you do if food is left out overnight?"
relevant_text = search_vector_database(query)
print(f"### Related Content:\n\n{relevant_text}\n\n")

chat_model = GenerativeModel("gemini-2.5-flash")  # Instantiate GenerativeModel directly
chat = chat_model.start_chat()

response = chat.send_message(
    f"Use the following to answer the question:\n\n{relevant_text}\n\nQuestion: {query}"
)
print(f"### Model Response:\n\n{response.text}")

FailedPrecondition: 400 Missing vector index configuration. Please create the required index with the following gcloud command: gcloud firestore indexes composite create --project=qwiklabs-gcp-01-32c593469378 --collection-group=food-safety --query-scope=COLLECTION --field-config=vector-config='{"dimension":"768","flat": "{}"}',field-path=embedding

In [21]:
!gcloud firestore indexes composite create \
--project=qwiklabs-gcp-01-32c593469378 \
--collection-group=food-safety \
--query-scope=COLLECTION \
--field-config=vector-config='{"dimension":"768","flat": "{}"}',field-path=embedding

Create request issued
Created index [CICAgOjXh4EK].


In [22]:
# Run the command again after the index has been created.
search_vector_database("How should I store food?")

' Store foods away from dripping condensate , at least six inches above the floor and with enough space between items to encourage air circulation. Freezer Storage Freezing is an excellent method for prolonging the shelf life of foods. By keeping foods frozen solid, the bacterial growth is minimal at best. However, if frozen foods are thawed and then refrozen, then harmful bacteria can reproduce to dangerous levels when thawed for the second time. In addition to that, the quality of the food is also affected. Never refreeze thawed foods, instead use them immediately. Keep the following rules in mind for freezer storage:  Use First In First Out method of stock rotation. All frozen foods should be frozen solid with temperature at 0°F or lower. Always use clean containers that are clearly labeled and marked, and have proper and secure lids. Allow adequate spacing between food containers to allow for proper air circulation. Never use the freezer for cooling hot foods. * * Tip: When receivi

In [24]:
search_vector_database("What should you do if food is left out overnight?")

'Furthermore, it is improper to store food in ice machines or ice that will be later used for human consumption. Food should be stored at least six inches off the floor, away from walls and dripping pipes. Keep all food, bulk or otherwise, covered and safe from contamination. Check food daily and throw away any spoiled or contaminated food. Store cleaning, disinfecting, and other chemicals away from foods, clearly marked and in their original containers. Keep food refrigerated at a temperature of 41°F or below. Monitor temperatures regularly with a thermometer placed in the warmest part of the refrigerator. Keep all cooling compartments closed except when you are using them. Store food in a refrigerator in such a way that the air inside can circulate freely. Keep all refrigerated foods covered, and use up stored leftovers quickly. When dishes and utensils are sparkling clean, keep them that way by proper storage.\n\nThis will cause the temperature  of the refrigerator to rise and other

# Task 4. Deploy a Generative AI application to search your vector store
Now that your vector database is prepared, in this section you will work on the client application to query it and return answers generated by Gemini.

**Please refer to the workings.txt file for the remaining steps of this lab**